In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker
from astropy.io import ascii

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord, fill_values_by_query
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import show_images
from SAGA.objects.object_catalog import calc_fiducial_p_sat, calc_fiducial_p_sat_corrected

from matplotlib.transforms import Bbox

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    
    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

base = annotate_base(base)

In [ ]:
nsat_missing_all = int(np.ceil(Query(~C.has_spec, C.faint_end_limit).filter(base, "p_sat_corrected").sum()))
nsat_missing_limit = int(np.ceil(Query(~C.has_spec, C.r_abs_limit).filter(base, "p_sat_corrected").sum()))
print(nsat_missing_all, nsat_missing_limit)

In [ ]:
sats = C.is_sat.filter(base)
sats_maybe_all = Query(~C.has_spec, "p_sat_corrected > 0").filter(base)

In [ ]:
seed = 1177 #923 #444 #177 # 114 # 47
while True:
    #seed += 1
    sats_maybe = Query(
        (lambda p: p > np.random.RandomState(seed).rand(len(p)), "p_sat_corrected"),  # must put in 1st place
        ~C.has_spec, 
    ).filter(base)

    if (np.abs((C.faint_end_limit.count(sats_maybe) - nsat_missing_all) < 1.5 and
        C.r_abs_limit.count(sats_maybe)) == nsat_missing_limit and
        (~Query(C.high_priority_sb_tight, C.gr_cut_tight, "Mr > -15.5")).count(sats_maybe) == 0):
        print(seed)
        break
    else:
        seed += 1
        
print(C.r_abs_limit.count(sats_maybe), len(sats_maybe))

In [ ]:
assert len(sats) == 127
assert C.r_abs_limit.count(sats) == 123
assert Query("EW_Halpha < 2").count(sats) == 18
assert Query("EW_Halpha < 2", C.r_abs_limit).count(sats) == 18
assert Query("EW_Halpha < 2", "Mr < -16").count(sats) == 2
assert len(np.unique(sats["HOSTID"])) == 34

In [ ]:
is_p1_sats = Query(C.is_sat, saga.host_catalog.construct_host_query("paper1_complete"), QueryMaker.in1d("OBJID", [4443380000000000475, 3115540000000003218], invert=True))

In [ ]:
lg = SAGA.database.FitsTable("../data/McConnachie2012.fit").read()
lg["Mr"] = lg["vmag_lc"] - 0.4
#lg["SubG"][np.argmax(lg["Name"] == "Pegasus dIrr")] = "M31 "
#lg = (Query((np.isfinite, "vmag_lc"), ~QueryMaker.equals("Name", "Canis Major"))).filter(lg)

mw_sat_mr = np.asarray(np.sort(QueryMaker.equals("SubG", "MW  ").filter(lg, "Mr")))
m31_sat_mr = np.asarray(np.sort(QueryMaker.equals("SubG", "M31 ").filter(lg, "Mr")))

In [ ]:
lg

In [ ]:
lg["sb_v"] = lg["Vmag"] + 2.5 * np.log10(2 * np.pi * (1-lg["Ell"]) * (lg["R1"] * 60)**2)
lg["sb_r"] = lg["Vmag"] - 0.4 + 2.5 * np.log10(2 * np.pi * (1-lg["Ell"]) * (lg["R1"] * 60)**2)

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [ ]:
plt.scatter(sats["Mr"], sats["sb_r"], c="C2", s=11, alpha=0.8, label="SAGA satellites")
plt.scatter(sats_maybe["Mr"], sats_maybe["sb_r"], marker='o', c='None', edgecolors='k', s=11, lw=0.5)
plt.scatter(lg["Mr"], lg["sb_r"], c="C1", marker="*", s=17, label="Local Group dwarfs\n(McConnachie 2012)")

plt.axvspan(-12.3, 0, color="grey", alpha=0.5)
plt.ylim(31, 16)
plt.xlim(-1, -20)

ax = plt.gca()
ax.set_xticks(np.arange(-2, -21, -2))
ax.set_xticks(np.arange(-1, -20.4, -0.5), minor=True)

ax.set_yticks(np.arange(30, 15, -2))
ax.set_yticks(np.arange(31, 15.6, -0.5), minor=True)

plt.text(-16.14, 30.2, "$\\leftarrow$     SAGA range    $\\rightarrow$", ha="center", fontsize=14)
plt.xlabel("$M_{r,o}$")
plt.ylabel("$\\mu_{{\\rm eff}, r_o}$  [mag arcsec$^{-2}$]")
plt.legend(loc="upper left", fontsize=14, handletextpad=0.1)

In [ ]:
Query("Mr < -12.3", "sb_r > 25.1").filter(lg)

In [ ]:
Query("Mr < -12.3").filter(lg)

In [ ]:
3/37*123

In [ ]:
plt.hist(Query("Mr < -15").filter(sats_this, 'log_sm'))

In [ ]:
SAGA.database.GoogleSheets("1O8tGgnHXRcAT8P78J3V2pWVb3CWE1Hvcowily79tdg4", 0).read()